# Финальный проект: корпус дневников (часть 2)
### Авторы проекта: Вероника Ганеева, Алла Горбунова, Елизавета Клыкова (БКЛ181)

## Шаг 3: поиск в корпусе

In [1]:
# %load_ext pycodestyle_magic
# %pycodestyle_on

In [2]:
import re
import ast
import pandas as pd
from pymystem3 import Mystem
from pprint import pprint
from nltk.tokenize import sent_tokenize
from pymorphy2 import MorphAnalyzer
pm = MorphAnalyzer()

Считываем корпус из файла, преобразуем в pandas dataframe, записываем колонки, данные которых понадобятся при выдаче, в переменные. Строки с разборами преобразуем в списки с помощью модуля ast.

Поскольку каждому тексту соответствует ровно одна строка датафрейма, возможно обращение к элементам этого текста по индексам списков.

In [3]:
def open_corpus(filename):
    corpus = pd.read_csv(filename, sep='\t')
    links = list(corpus['link'])
    post_ids = list(corpus['post_id'])
    authors = list(corpus['author'])
    titles = list(corpus['title'])
    texts = list(corpus['text'])
    sentences = [ast.literal_eval(item)
                 for item in corpus['sentences'].to_list()]
    parsed_texts = [ast.literal_eval(item)
                    for item in corpus['parsed_text'].to_list()]
    sent_texts = [ast.literal_eval(item)
                  for item in corpus['sent_text'].to_list()]
    corpus_volume = 0
    for text in parsed_texts:
        corpus_volume += len(text)
    volume = 'текстов - {}, словоформ - {}.'.format(len(texts), corpus_volume)
    welcome = '''Добро пожаловать в корпус дневников! \
На сегодняшний день в нем содержится: '''
    welcome_note = welcome + volume
    return links, post_ids, authors, titles, texts, sentences, parsed_texts, sent_texts, welcome_note

In [4]:
links, post_ids, authors, titles, texts, sentences, parsed_texts, sent_texts, welcome_note = open_corpus('corpus.tsv')

Здесь и далее длинные строки оставлены там, где не получилось их перенести без поломки программы.

### Обработка запроса

In [5]:
TAGS = ['NOUN', 'ADJ', 'VERB', 'ADV', 'PREP', 'PRCL', 'PRON', 'ANUM', 'CONJ', 'INTJ', 'NUM', 'COM']

In [6]:
# возвращает все возможные леммы для формы
def get_lemmas(word):
    variants = pm.parse(word)
    lemmas = list(set(map(lambda x: x.normal_form, variants)))
    return lemmas

In [7]:
# возвращает параметры одного слова
def get_params(q_word):
    params = {'form' : None,
              'pos' : None,
              'lemma' : None}
    if '"' in q_word:
        params['form'] = q_word.strip('"')
    elif q_word.upper() in TAGS:
        params['pos'] = q_word.upper()
    elif '+' in q_word:
        parts = q_word.split('+')
        params['lemma'] = [parts[0].lower()]
        params['pos'] = parts[1].upper()
    else:
        params['lemma'] = get_lemmas(q_word) 
    return params

In [8]:
# проверка запроса на соответствие требованиям
def correct(query):
    #if len(query.split()) > 7:
    #    raise Exception('Ваш запрос содержит больше семи слов.')
    if '\'' in query:
        raise Exception('Пожалуйста, используйте двойные кавычки.')
    if len(query) < 1 or query == ' ':
        raise Exception('Вы ввели пустой запрос.')
    if re.findall('[^а-яёА-ЯЁa-zA-Z+ "-]', query) != []:
        raise Exception('Вы используете недопустимые символы.')
    for item in re.findall('[a-zA-Z]+', query):
        if item.upper() not in TAGS:
            raise Exception('Недопустимый тег: %s' % item)

Итоговая функция обработки запроса:

In [9]:
def parse_query(query):
    correct(query)
    split_query = query.split()  # узнаем кол-во слов
    if len(split_query) == 1:
        params = [get_params(split_query[0])]
    else:
        params = []
        for q in split_query:
            params.append(get_params(q))
    return params

Функция поиска:

In [10]:
def search_corpus(list_of_search, sent_texts=sent_texts):
    find_all = ['begin']
    for search_word in list_of_search:
        if find_all == ['begin']:
            for num_text, text in enumerate(sent_texts):
                for num_sent, sent in enumerate(text):
                    for num_word, word in enumerate(sent):
                        if (search_word['form'] is None or search_word['form'] == word[0]) and (search_word['lemma'] is None or word[1] in search_word['lemma']) and (search_word['pos'] is None or search_word['pos'] == word[2]):
                            find_all
                            find_all.append([num_text, num_sent, num_word])
            find_all.remove('begin')
        elif find_all != []:
            new_findall = []
            for el in find_all:
                if len(sent_texts[el[0]][el[1]]) > el[2] + 1:
                    word_from_sent = sent_texts[el[0]][el[1]][el[2] + 1]
                    if (search_word['form'] is None or search_word['form'] == word_from_sent[0]) and (search_word['lemma'] is None or word_from_sent[1] in search_word['lemma']) and (search_word['pos'] is None or search_word['pos'] == word_from_sent[2]):
                        new_findall.append([el[0], el[1], el[2] + 1])
            find_all = new_findall
    return find_all

Собираем функции, добавляем получение метаданных:

In [11]:
def beautiful_answer(findall, links=links, post_ids=post_ids, authors=authors, titles=titles,  sentences=sentences, sent_texts=sent_texts):
    if findall == []:
        print('В корпусе не нашлось такого сочетания.')
    else:
        all_results = []
        for el in findall:
            list_to_print = {}
            list_to_print['Предложение'] = sentences[el[0]][el[1]]
            list_to_print['Автор'] = authors[el[0]]
            if str(titles[el[0]]) != 'nan':
                list_to_print['Название'] = titles[el[0]]
            else:
                list_to_print['Название'] = 'Без названия'
            list_to_print['id текста'] = post_ids[el[0]]
            list_to_print['Ссылка'] = links[el[0]]
            all_results.append(list_to_print)
        all_results = pd.DataFrame(all_results)
        return all_results

## Поиск в корпусе
### Инструкция для пользователя
*Для работы с корпусом запустите программу с помощью Restart & Run All, затем введите запрос в открывшееся в последней ячейке поле и нажмите Enter.*

Пожалуйста, используйте только частеречные теги из данного списка:

*NOUN - существительное  
ADJ - прилагательное  
VERB - глагол  
ADV - наречие  
NUM - числительное  
ANUM - числительное-прилагательное  
PRON - местоимение   
CONJ - союз  
PREP - предлог  
PRCL - частица  
INTJ - междометие  
COM - часть сложного слова*  

1. Точную форму слова нужно указывать в двойных кавычках (вот таких: " ").
2. B запросах вида *Х+тег* Х должен быть начальной формой слова.
3. Поиск точной формы чувствителен к регистру.
4. Частеречные теги не чувствительны к регистру.
5. Если Вы хотите найти несколько слов в точной форме, пожалуйста, берите в кавычки отдельно каждое слово.
6. Не используйте в поисковом запросе цифры и символы, кроме знака +.

In [12]:
print(welcome_note)

Добро пожаловать в корпус дневников! На сегодняшний день в нем содержится: текстов - 133, словоформ - 32705.


In [14]:
list_of_search = parse_query(input())
df_result = beautiful_answer(search_corpus(list_of_search))
pd.set_option('display.max_colwidth', -1)
df_result

печь+noun


,Предложение,Автор,Название,id текста,Ссылка
0,Мёртвой нищей логовище без печей.,Tay-Li,Старая усадьба,220050375,https://tay-li.diary.ru/p220050375_staraya-usadba.htm
